In [1]:
!pip install requests

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install pandas

In [4]:
!pip install lxml

In [5]:
!pip install selenium

In [23]:
!pip install python-dotenv

In [6]:
years = range(1994, 2024)

In [7]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [12]:
import requests

for year in years: 
    url = url_start.format(year)
    data = requests.get(url)
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [13]:
# import pandas as pd
# from io import StringIO
# from bs4 import BeautifulSoup

# with open("mvp/1994.html") as f:
#     page = f.read()

# soup = BeautifulSoup(page, "html.parser")

# # Removing extra header
# soup.find('tr', class_="over_header").decompose()

# # Removing everything else from the page outside of the mvp table we want
# mvp_table = soup.find(id="mvp")

# mvp_1994 = pd.read_html(StringIO(str(mvp_table)))

# # Displays single data frame
# mvp_1994[0]

In [14]:
# Parsing all of the tables
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(StringIO(str(mvp_table)))[0]

    # Adding a year column
    mvp["Year"] = year
    
    dfs.append(mvp)

In [15]:
# Combining all of the dataframes
mvps = pd.concat(dfs)

In [16]:
mvps.to_csv("mvps.csv")

In [17]:
# Looking at stats for all players in the same year range

player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
# This page uses javascript to render all of the rows because of this we need to use selenium

In [24]:
from dotenv import load_dotenv
import os
from selenium import webdriver

load_dotenv()

True

In [25]:
driver_url = os.getenv("DRIVER_PATH")
driver = webdriver.Chrome(executable_path=driver_url)

C:\Users\Caleb\AppData\Local\Temp\ipykernel_48076\2442837788.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_url)
